In [5]:
#dependencies
import requests
import pandas as pd 
from config import fred_key, nasdaq_key #, bls_key
import json
import hvplot.pandas
import matplotlib.pyplot as plt
import numpy as np
# import folium   
# from folium import plugins
import traceback
import nasdaqdatalink
nasdaqdatalink.ApiConfig.api_key = nasdaq_key
import os
# API keys obtained from  https://fred.stlouisfed.org/docs/api/fred/ and https://www.bls.gov/developers/home.htm and https://data.nasdaq.com/ using nasdaqdatalink lib is a bit easier, documentation is here https://pypi.org/project/Nasdaq-Data-Link/

In [7]:

# IDs of different data sets in percentages, using MORTGAGE observation as the other ones do not look relatable
# MORTGAGE30US  Interest rate in percentage per year. 
# EXHOSLUSM495S Existing home sales 
# GINIALLRH  Income Gini Ratio for Households by Race of Householder, All Races
# GINIALLRH Income Gini Ratio for Households by Race of Householder, All Races
# T10YIE Inflation
# UNRATE UNEMPLOYMENT RATE. 
# RIFLGFLXIINA reasury Long-Term Average (Over 10 Years), Inflation-Indexed
filePath = "./Resources/mortgage_interest_rates.csv"
if not os.path.isfile(filePath):
    url = "https://api.stlouisfed.org/fred/series/observations?series_id=MORTGAGE30US&observation_start=2010-01-01&observation_end=2021-12-31&frequency=m&file_type=json&api_key=" + fred_key
    res = requests.get(url)
    fred_data = json.loads(res.text)  #reuturns the interest rate per year 
    fred_data = pd.DataFrame.from_dict(fred_data['observations'])
else:
    fred_data = pd.read_csv(filePath)
# interest Rate mortgate per year need to get the MEAN per year to match the data from 2010 to 2021 and make a correlation against other sets like inflation, and interest rates for the graphs.  

fred_data.to_csv(filePath)
fred_data

,Unnamed: 0,realtime_start,realtime_end,date,value
0,0,2023-03-30,2023-03-30,2010-01-01,5.03
1,1,2023-03-30,2023-03-30,2010-02-01,4.99
2,2,2023-03-30,2023-03-30,2010-03-01,4.97
3,3,2023-03-30,2023-03-30,2010-04-01,5.10
4,4,2023-03-30,2023-03-30,2010-05-01,4.89
...,...,...,...,...,...
139,139,2023-03-30,2023-03-30,2021-08-01,2.84
140,140,2023-03-30,2023-03-30,2021-09-01,2.90
141,141,2023-03-30,2023-03-30,2021-10-01,3.07
142,142,2023-03-30,2023-03-30,2021-11-01,3.07


In [8]:
# BLS.GOV API 
# 'CUUR0000SA0',  onsumer Price Index - All Urban Consumers
# 'SUUR0000SA0',  Chained CPI-All Urban Consumers
#  'CEU0800000003',  Employment & Unemployment
#  'APU000070111',  Inflation & Prices
# FPCPITOTLZGUSA  Inflation, consumer prices for the United States 

# 5-Year 0.625% Treasury Inflation-Indexed Note, Due 4/15/2023 (DTP5A23)


''' 
After checking various data sets, It think that creating a correlaton of inflation vs interes rates or 

Interest rates and inflation tend to move in the same direction – when inflation is increasing, 
banks will increase interest rates to encourage people to spend less and save more.
In theory, this should reduce demand for goods and services, which helps to contain inflation.

How are inflation and unemployment related?
Inflation has historically had an inverse relationship with unemployment.
This means that when inflation rises, unemployment drops. Higher unemployment, on the other hand, equates to lower inflation.
'''
# res_json['Results']['series'][0]['data']  so [x for x in res_json['Results']['series'][x]['data] ]
'''
What is the Consumer Price Index? The Consumer Price Index (CPI), produced by the Bureau of Labor Statistics (BLS), 
is the most widely used measure of inflation. The primary CPI (CPI-U) .
is designed to measure price changes faced by urban consumers, who represent 93% of the U.S. population.

'''
filePath2 = "./Resources/mortgage_interest_rates.csv"
if not os.path.isfile(filePath2):
    data = json.dumps({"seriesid": ['CEU0800000003'],"startyear":"2010", "endyear":"2021"}) #
    response = requests.post('https://api.bls.gov/publicAPI/v2/timeseries/data/', data=data, headers={'Content-type': 'application/json'})
    res_json = json.loads(response.text)
    bls_pd = pd.DataFrame([x for x in res_json['Results']['series'][0]['data']])   
    bls_pd.drop(['period', 'footnotes'], axis=1, inplace=True)
else:
    bls_pd = pd.read_csv(filePath2)

bls_pd.to_csv("./Resources/mortgage_interest_rates.csv")
bls_pd

,Unnamed: 0.1,Unnamed: 0,realtime_start,realtime_end,date,value
0,0,0,2023-03-30,2023-03-30,2010-01-01,5.03
1,1,1,2023-03-30,2023-03-30,2010-02-01,4.99
2,2,2,2023-03-30,2023-03-30,2010-03-01,4.97
3,3,3,2023-03-30,2023-03-30,2010-04-01,5.10
4,4,4,2023-03-30,2023-03-30,2010-05-01,4.89
...,...,...,...,...,...,...
139,139,139,2023-03-30,2023-03-30,2021-08-01,2.84
140,140,140,2023-03-30,2023-03-30,2021-09-01,2.90
141,141,141,2023-03-30,2023-03-30,2021-10-01,3.07
142,142,142,2023-03-30,2023-03-30,2021-11-01,3.07


In [9]:
# Gross domestic product is the monetary value of all finished goods and services made within a country during a specific period.
# GDP provides an economic snapshot of a country, used to estimate the size of an economy and its growth rate

filePath3 = "./Resources/gdp.csv"
if not os.path.isfile(filePath3):
    data = nasdaqdatalink.get("FRED/GDP", start_date="2010-01-01", end_date="2022-12-31", collapse="yearly")
    gdp_df = pd.DataFrame(data)
    gdp_df.to_csv("./Resources/gdp.csv")
else:
    gdp_df = pd.read_csv(filePath3)
# WE HAVE TO GROUPBY MEAN PER YEAR SO WE GET THE AVERAGE PER YEAR FROM 2010 to 2021 which is all that is available 
gdp_df

,Value
Date,
2010-01-01,14764.611
2010-04-01,14980.193
2010-07-01,15141.605
2010-10-01,15309.471
2011-01-01,15351.444
2011-04-01,15557.535
2011-07-01,15647.681
2011-10-01,15842.267
2012-01-01,16068.824


In [12]:

# res
''' 

HOME VALUES INDICATOR we are using is the Single Family Home value ZSFH 

XHVI All Home - Top tier 
The USA region code will always be 102001

'''
filePath4 = "./Resources/zillosSFH.csv"
if not os.path.isfile(filePath4):
    zurl = "https://data.nasdaq.com/api/v3/datatables/ZILLOW/DATA?indicator_id=ZSFH&region_id=102001&api_key=" + nasdaq_key
    zres = requests.get(zurl)
    zjson = json.loads(zres.text)
    zjson_pd = pd.DataFrame(zjson)
    zjson_pd.to_csv("./Resources/zillosSFH.csv")
else:
    zjson_pd = pd.read_csv(filePath4)
#  We need to use pandas to extract from 2010 to 2021 and MEAN per year to match the data sets and make it all look prety, create a plot, maybe a line plot etc... 

# Can an index be averaged? I wondered if this was mathematically legal and seems like it is per google. 

# Average Index Value means the sum of the monthly index values during the 12-month period ending in October 31 divided by 12
#  (in the case of indices published monthly) or the sum of the bi-monthly index values divided by 6 (in the case of indices published bi-monthly).
zjson_pd

,Unnamed: 0,datatable,meta
0,data,"[['ZSFH', '102001', '2023-02-28', 327390.07546...",NaN
1,columns,"[{'name': 'indicator_id', 'type': 'text'}, {'n...",NaN
2,next_cursor_id,NaN,NaN


In [13]:
filePath5 = "./Resources/unemployment.csv"
if not os.path.isfile(filePath5):
    unemploymentRateData = nasdaqdatalink.get("FRED/UNRATE", start_date="2001-12-31", end_date="2005-12-31")
    # unemploymentRateData
    unplmnt_pd = pd.DataFrame(unemploymentRateData)
else:
    unplmnt_pd = pd.read_csv(filePath5)
unplmnt_pd

,Value
Date,
2002-01-01,5.7
2002-02-01,5.7
2002-03-01,5.7
2002-04-01,5.9
2002-05-01,5.8
2002-06-01,5.8
2002-07-01,5.8
2002-08-01,5.7
2002-09-01,5.7


# TEST TEST TEST